In [3]:
%pip install optbinning pandas-profiling -U -q

import pandas as pd
import numpy as np

Note: you may need to restart the kernel to use updated packages.


In [4]:
transformed_data = pd.read_parquet('transformed_data.parquet')
holding_matrix = pd.read_parquet('holding_matrix.parquet')

In [10]:
transformed_data = transformed_data.rename(columns={k: f'xsold_{k}' for k in ['Casualty', 'Corporate', 'Cyber',
       'DM and Partnership', 'Environmental Risks', 'Financial Lines', 'Fire',
       'Marine', 'Property', 'Tech Lines', 'Terrorism', 'Travel']})

In [11]:
sdf = pd.concat([transformed_data, holding_matrix], axis=1)

In [12]:
from pandas_profiling import ProfileReport
profile = ProfileReport(sdf, title="Univariate Minimal Report", minimal=True)

profile.to_file("minimal_report.html")

import json
json_data = json.loads(profile.to_json())

keys = ['variable_name', 'n_distinct', 'p_distinct', 'is_unique', 'n_unique', 'p_unique', 'type', 'hashable', 'ordering', 'n_missing', 'n', 'p_missing', 'count', 
 'memory_size', 'n_negative', 'p_negative', 'n_infinite', 'n_zeros', 'mean', 'std', 'variance', 'min', 'max', 'kurtosis', 'skewness', 'sum', 
 'mad', 'range', '5%', '25%', '50%', '75%', '95%', 'iqr', 'cv', 'p_zeros', 'p_infinite', 'monotonic_increase', 'monotonic_decrease', 
 'monotonic_increase_strict', 'monotonic_decrease_strict', 'monotonic', 'top_ten_value_counts_without_nan', 'bottom_ten_value_counts_without_nan']

profile_dict = {k:[] for k in keys}
for i in json_data['variables'].keys():
    for j in keys:
        try:
            if j=='variable_name':
                profile_dict['variable_name'].append(i)
            elif j=='top_ten_value_counts_without_nan':
                top_ten_value_counts_without_nan = list(sdf[i].value_counts().iloc[:10].index.astype(str))
                profile_dict[j].append(top_ten_value_counts_without_nan)
            elif j=='bottom_ten_value_counts_without_nan':
                bottom_ten_value_counts_without_nan = list(sdf[i].value_counts().iloc[-10:].index.astype(str))
                profile_dict[j].append(bottom_ten_value_counts_without_nan)
            else:
                profile_dict[j].append(json_data['variables'][i][j])
        except:
            profile_dict[j].append(np.nan)

pd.DataFrame(profile_dict).to_excel('univariate summary.xlsx', index=False)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Render JSON:   0%|          | 0/1 [00:00<?, ?it/s]

In [1]:
import time
time.sleep(300)